# Warsztaty badawcze - Projekt 2

## Real Estate Price Analysis - Model AutoML

### Wiktoria Boguszewska, Mateusz Zacharecki, Patrycja Żak


Uwaga: kod działa na colab lub lokalnie.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip uninstall -y mkl
!pip install -U pip
!pip install -U "mxnet<2.0.0"
!pip install autogluon
!pip install -U ipykernel

!pip uninstall -y autogluon.core
!pip uninstall -y autogluon
!pip install autogluon
# po instalacji trzeba zrestartować środowisko

Found existing installation: autogluon.core 1.1.1
Uninstalling autogluon.core-1.1.1:
  Successfully uninstalled autogluon.core-1.1.1
Found existing installation: autogluon 1.1.1
Uninstalling autogluon-1.1.1:
  Successfully uninstalled autogluon-1.1.1
  Using cached autogluon-1.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.core-1.1.1-py3-none-any.whl.metadata (11 kB)
Using cached autogluon-1.1.1-py3-none-any.whl (9.7 kB)
Using cached autogluon.core-1.1.1-py3-none-any.whl (234 kB)


In [2]:
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd

import random
random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('real_estate_ads 3.csv')
df.head()

,id,market,created_at_first,created_at,district_lon,district_lat,title,description,params,price,no_rooms,m,price_per_m,map_lon,map_lat
0,49341,secondary,2018-01-04 13:11:05,2018-01-08 11:27:35,16.9408,52.38356,NaN,<p>Zapraszam do zapoznania się z ofertą sprzed...,price<=>price<br>price<=>289000<br>price[curre...,289000.0,3,48.000000,6020.83,16.959516,52.391494
1,261720,secondary,2018-01-15 19:12:07,2018-01-25 11:11:31,16.9408,52.38356,NaN,<p>Zapraszam do zapoznania się z ofertą sprzed...,price<=>price<br>price<=>379000<br>price[curre...,379000.0,3,61.000000,6213.11,16.950743,52.380535
2,50441,secondary,2018-01-04 13:53:29,2018-01-11 05:55:12,16.9408,52.38356,Rataje Nowe Miasto Niski Blok,<p>Weychan Nieruchomości Monika Hałas - Kurcza...,price<=>price<br>price<=>339000<br>price[curre...,339000.0,3,63.000000,5380.95,16.945860,52.393188
3,203518,secondary,2018-01-12 11:10:57,2018-01-30 14:38:39,16.9408,52.38356,"Mieszkanie 3 pokojowe blisko tramwaju, Poznań os.",<p>Oferuję na sprzedaż jasne i zadbane mieszka...,price<=>price<br>price<=>330000<br>price[curre...,330000.0,3,59.200001,5574.32,16.980279,52.391129
4,5,secondary,2018-01-02 11:34:51,2018-01-18 14:22:29,16.9408,52.38356,"Inwestycja -rondo Rataje student AWF,POLITECHNIKA",<p><strong>Dwa pokoje na niskim piętrze tuż pr...,price<=>price<br>price<=>259000<br>price[curre...,259000.0,2,38.000000,6815.79,16.950027,52.395861


In [4]:
df = df.loc[df['market'] == 'secondary']

In [5]:
X, y = df.loc[:, df.columns != "price_per_m"], df["price_per_m"]

In [6]:
def find_optimal_bins(X, y, max_bins=10, test_size=0.2, random_state=42):
    best_q = 1
    best_diff = float('inf')

    for q in range(2, max_bins + 1):
        y_binned = pd.qcut(y, q=q, duplicates='drop')  # tworzymy koszyki
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y_binned
        )
        # Sprawdzamy różnicę między średnimi
        diff = abs(y_train.mean() - y_test.mean())

        # Zapisujemy najlepszą liczbę koszyków, gdzie różnica jest najmniejsza
        if diff < best_diff:
            best_diff = diff
            best_q = q

    return best_q

In [7]:
optimal_bins = find_optimal_bins(X, y, max_bins=20)
print(optimal_bins)

8


In [8]:
y_binned = pd.qcut(y, q=optimal_bins, duplicates='drop')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_binned)

In [10]:
train_combined = pd.concat([X_train, y_train], axis=1)

In [11]:
save_path = 'agModels-predictClass'
predictor = TabularPredictor(label='price_per_m', path=save_path, problem_type='regression', eval_metric='root_mean_squared_error').fit(train_combined, presets='best_quality')

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.10 GB / 12.67 GB (87.5%)
Disk Space Avail:   65.79 GB / 107.72 GB (61.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

(_ray_fit pid=5961) [1000]	valid_set's rmse: 322.094


(_ray_fit pid=6461) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=6461) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=6461)  [repeated 4x across cluster]
(_ray_fit pid=6461) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=6461) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=6461)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]


(_ray_fit pid=6461) [1000]	valid_set's rmse: 556.856 [repeated 2x across cluster]


(_ray_fit pid=6962) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster]
(_ray_fit pid=6962) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=6962)  [repeated 4x across cluster]
(_ray_fit pid=6962) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=6962) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=6962)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]
(_ray_fit pid=7341) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster]
(_ray_fit pid=7341) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=7341)  [repeated 4x across cluster]
(_ray_fit pid=7341) You can install it with `pip install dask[dataframe]` o

(_ray_fit pid=6962) [1000]	valid_set's rmse: 346.989 [repeated 2x across cluster]


(_ray_fit pid=7485) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
(_ray_fit pid=7485) Dask dataframe query planning is disabled because dask-expr is not installed.
(_ray_fit pid=7485)  [repeated 2x across cluster]
(_ray_fit pid=7485) You can install it with `pip install dask[dataframe]` or `conda install dask`.
(_ray_fit pid=7485) This will raise in a future version.
(_ray_fit pid=7485)   warnings.warn(msg, FutureWarning)


(_ray_fit pid=7341) [1000]	valid_set's rmse: 543.086
(_ray_fit pid=7485) [1000]	valid_set's rmse: 479.612


(_dystack pid=5221) 	-477.5642	 = Validation score   (-root_mean_squared_error)
(_dystack pid=5221) 	438.77s	 = Training   runtime
(_dystack pid=5221) 	2.36s	 = Validation runtime
(_dystack pid=5221) Fitting model: LightGBM_BAG_L1 ... Training model for up to 27.76s of the 295.78s of remaining time.
(_dystack pid=5221) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 16.18% memory usage per fold, 64.73%/80.00% total).
(_dystack pid=5221) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=16.18%)
(_ray_fit pid=8000) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
(_ray_fit pid=8000) Dask dataframe query planning is disabled because dask-expr is not installed.
(_ray_fit pid=8000) 
(_ray_fit pid=8000) You can install it with `pip install dask[dataframe]` or `conda install dask`.
(_ray_fit pid=8000) This will raise in a future ver

In [12]:
performance = predictor.evaluate(train_combined)
print(performance)

{'root_mean_squared_error': -178.60051130234177, 'mean_squared_error': -31898.142637457913, 'mean_absolute_error': -48.271381584774566, 'r2': 0.9846108324175528, 'pearsonr': 0.9924020825140382, 'median_absolute_error': -26.510390624999673}


In [13]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                   model    score_val              eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3  -292.498113  root_mean_squared_error      47.112190  1620.563036                0.000784           0.021112            3       True          8
1        LightGBM_BAG_L1  -296.822591  root_mean_squared_error       2.959165   223.684288                2.959165         223.684288            1       True          4
2    WeightedEnsemble_L2  -296.822591  root_mean_squared_error       2.959955   223.700274                0.000790           0.015987            2       True          5
3        LightGBM_BAG_L2  -298.231503  root_mean_squared_error      47.111406  1620.541924                1.765143         141.998804            2       True          7
4      LightGBMXT_BAG_L2  -347.003009  root_mean_squared_error      48.428483  2024.798981   

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L3': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': -1447.115331430354,
  'KNeighborsDist_BAG_L1': -1487.9099313090753,
  'LightGBMXT_BAG_L1': -432.8947643584917,
  'LightGBM_BAG_L1': -296.82259056756607,
  'WeightedEnsemble_L2': -296.82259056756607,
  'LightGBMXT_BAG_L2': -347.0030085502424,
  'LightGBM_BAG_L2': -298.231503311675,
  'WeightedEnsemble_L3': -292.49811251497636},
 'model_best': 'WeightedEnsemble_L3',
 'model_paths': {'KNeighborsUnif_BAG_L1': ['KNeighborsUnif_BAG_L1'],
  'KNeighborsDist_BAG_L1': ['KNeighborsDist_BAG_L1'],
  'LightGBMXT_BAG_L1': ['LightGBMXT_

In [14]:
test_combined = pd.concat([X_test, y_test], axis=1)

In [15]:
rmse_score = predictor.evaluate(test_combined)
print("RMSE na zbiorze testowym:", rmse_score)

RMSE na zbiorze testowym: {'root_mean_squared_error': -271.75972842365303, 'mean_squared_error': -73853.34999289765, 'mean_absolute_error': -78.42764231902026, 'r2': 0.965265696657907, 'pearsonr': 0.9830622817865857, 'median_absolute_error': -37.987031249999745}


In [16]:
print("RMSE:", round(abs(rmse_score['root_mean_squared_error']),2))

RMSE: 271.76


RMSE: 271.76